In [26]:
# -*- coding: <utf-8> -*-
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import unidecode
import csv 
import io




In [38]:
train_fname = 'train.csv'
test_fname = 'test.csv'
X = []
y = []
with io.open(train_fname, encoding = 'utf-8') as f:
     for line in f: 
        line = line.replace("\\xa0", "")
        line = line.replace("\\n", "")
        line = line.lower()
        y.append(int(line[0]))
        X.append(line[5:-6])
y = np.array(y)



print('n_samples : %d' % len(X))

n_samples : 4415


In [39]:
print("Le taux de prédiction au niveau de la chance est : %s" % np.mean(y == 0))

Le taux de prédiction au niveau de la chance est : 0.694450736127


In [62]:
wordContent = []
wordCount = dict()
import re
regEx = re.compile('\\W*') # match any ~[A-Z0-9]
for x in X:
    for word in regEx.split(x):
        if word in wordCount.keys():
            wordCount[word] = wordCount[word] + 1
        else:
            wordCount[word] = 1



In [82]:
stopList = set()
with io.open('stop_word.txt', newline = '\n') as f:
     for line in f: 
        line = line.strip()
        stopList.add(line)
print stopList

set([u'all', u"she'll", u"don't", u'being', u'over', u'through', u'yourselves', u'its', u'before', u"he's", u"when's", u"we've", u'had', u'should', u"he'd", u'to', u'only', u"there's", u'those', u'under', u'has', u"haven't", u'do', u'them', u'his', u"they'll", u'very', u"who's", u"they'd", u'cannot', u"you've", u'they', u'not', u'during', u'yourself', u'him', u'nor', u"we'll", u'did', u"they've", u'this', u'she', u'each', u"won't", u'where', u"mustn't", u"isn't", u"i'll", u"why's", u'because', u"you'd", u'doing', u'some', u'up', u'are', u'further', u'out', u'what', u'for', u'while', u"wasn't", u'does', u"shouldn't", u'above', u'between', u'be', u'we', u'who', u"you're", u'were', u'here', u'hers', u"aren't", u'by', u'both', u'about', u'would', u'of', u'could', u'against', u"i'd", u"weren't", u"i'm", u'or', u"can't", u'own', u'into', u'whom', u'down', u"hadn't", u"couldn't", u'your', u"doesn't", u'from', u"how's", u'her', u'their', u"it's", u'there', u'been', u'why', u'few', u'too', u'th

In [83]:
def filtre(wordCount, cirtere, stopList):
    wordList = []
    for key, value in wordCount.iteritems():
        if key not in stopList and value > cirtere:
            wordList.append(key)
    return wordList
            
        
    
def words2Vectors(wordList, data):
    vec = [0] * len(wordList)
    for word in data:
        if word in wordList:
            vec[wordList.index(word)] = vec[wordList.index(word)] + 1 // +1
    return vec

def createWordList(wordContent):
    # no word occur twice
    wordSet = set()
    for word in wordContent:
        if word not in wordSet:
            wordSet.add(word)
    return list(wordSet)

In [58]:
wordList  = createWordList(wordContent)

In [86]:
wordList = filtre(wordCount, 2, stopList)
vectors = []
for x in X:
    vectors.append(words2Vectors(wordList, x))

AttributeError: 'list' object has no attribute 'shape'

In [89]:
with open('vectors.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(vectors)

In [91]:
print wordList

[u'', u'yellow', u'four', u'mujh', u'hanging', u'lord', u'immature', u'callin', u'foul', u'screaming', u'commenter', u'solid', u'asian', u'nigh', u'tired', u'second', u'xf6r', u'errors', u'thunder', u'hero', u'reporter', u'brainwashed', u'reported', u'china', u'kids', u'k', u'reports', u'menu', u'military', u'criticism', u'projection', u'follo', u'brought', u'cocaine', u'unit', u'opponents', u'spoke', u'dnc', u'aur', u'music', u'telegraph', u'strike', u'females', u'holy', u'relax', u'successful', u'brings', u'hurt', u'90', u'hole', u'hold', u'couldn', u'locked', u'u1ebf', u'example', u'august', u'organized', u'want', u'counseling', u'absolute', u'hoe', u'poverty', u'complaining', u'hoo', u'travel', u'damage', u'hot', u'gaming', u'wrong', u'types', u'effective', u'wins', u'keeps', u'democratic', u'wing', u'wind', u'feedback', u'reforms', u'fit', u'fix', u'easier', u'effects', u'schools', u'silver', u'represents', u'zinya', u'financial', u'series', u'solution', u'ring', u'rs', u're', u'e